# Knochentypen

Zuerst gruppieren wir die Daten nach Knochentyp, weil wir nur innerhalb dieser Gruppen valide Vergleiche machen können.

In [1]:
import utils.load_write as lw
import utils.datastructure_operations as dataops

df_horse = lw.load_csv("data/csv/abmap_horse.csv")

element_grp = df_horse.groupby(['ELEMENT'])
#print(df.ELEMENT.nunique(), "distinkte Einträge")
print("Es existieren Funde zu", element_grp.ngroups, "verschiedenen Knochentypen.")
list(element_grp.groups.keys())

Es existieren Funde zu 14 verschiedenen Knochentypen.


['Astragalus',
 'Calcaneum',
 'Femur',
 'First phalanx',
 'Humerus',
 'Mandible',
 'Metacarpal',
 'Metatarsal',
 'Pelvis',
 'Radius',
 'Scapula',
 'Second phalanx',
 'Tibia',
 'Ulna']

Nun aktualisieren wir für jeden Datenpunkt die jeweilige Datierung (Periode), sodass wir nur noch mit den vier zuvor festgelegten, gröberen Perioden arbeiten. Dann zählen wir, aus wie vielen dieser Perioden für jeden Typ Funde vorliegen, denn wenn nur Funde aus einer Periode vorliegen, eignet sich der entsprechende Knochentyp natürlich nicht für einen Vergleich über die Zeit.
Außerdem zählen wir, wie viele verschiedene Knochen jeweils gefunden wurden und wie viele Messungen pro Typ gemacht wurden, denn auch hier gilt: Zu wenige Daten liefern keine aussagekräftigen Ergebnisse.

In [2]:
import pandas as pd
time_ranges_condensed = lw.load_json("data/json/time-ranges-condensed.json")

relevant_bone_types = list(element_grp.groups.keys())
bone_types_dict = dict()

for bone_type in relevant_bone_types:
    ''' 'PERIOD' updaten und alle Spalten entfernen, die nicht benötigt werden '''
    element_dataframe = dataops.update_period(element_grp.get_group(bone_type), time_ranges_condensed)

    ''' save the data for each type to files for better overview '''
    #path = "data/csv/elements/" + bone_type.lower().replace(" ","_")
    #lw.save_csv(element_dataframe, path + ".csv")

    ''' count the amount of different bones for this type '''
    ctr = len(dataops.count_amounts_in_column(element_dataframe, "BONEID"))

    ''' count the amount of periods for this type '''
    periods = len(dataops.count_amounts_in_column(element_dataframe, "PERIOD"))

    bone_types_dict.update(dataops.bone_type_dict(bone_type, element_dataframe, ctr, periods))

Die Anzahl der Funde und Messungen pro Knochentyp und auch pro Periode sollen als Barchart abgebildet werden.
Zur einfacheren Weiterverarbeitung werden diese Daten daher in Listen gespeichert.

In [3]:
''' Lists for barchart data '''
bone_amounts = []
meas_amounts = []

df_bone_types = pd.DataFrame(columns=['BONEID', 'MEASURE', 'MEASTYPE', 'PERIOD', 'RANGE'])

for key in list(bone_types_dict.keys()):
    bones = bone_types_dict[key]["bone_amt"]
    bone_amounts.append(bones)
    meas = bone_types_dict[key]["meas_amt"]
    meas_amounts.append(meas)
    periods = bone_types_dict[key]["periods_amt"]

    print(key, ":", bones, "Fund(e) und", meas, "Messungen. Datierungen für", periods, "Perioden.")

    ''' Remove Types that don't have enough data to support any statistics about them '''
    if bones < 10 or periods <= 2:
        print("- DELETING", key, " - not enough data")
        del bone_types_dict[key]
        relevant_bone_types.remove(key)
    else:
        df_bone_types = pd.concat([df_bone_types, bone_types_dict[key]["df"]])


#lw.save_json(relevant_bone_types, "data/json/relevant-bone-types.json")

Astragalus : 81 Fund(e) und 229 Messungen. Datierungen für 3 Perioden.
Calcaneum : 33 Fund(e) und 46 Messungen. Datierungen für 3 Perioden.
Femur : 51 Fund(e) und 108 Messungen. Datierungen für 3 Perioden.
First phalanx : 134 Fund(e) und 540 Messungen. Datierungen für 3 Perioden.
Humerus : 97 Fund(e) und 208 Messungen. Datierungen für 3 Perioden.
Mandible : 1 Fund(e) und 4 Messungen. Datierungen für 1 Perioden.
- DELETING Mandible  - not enough data
Metacarpal : 112 Fund(e) und 455 Messungen. Datierungen für 3 Perioden.
Metatarsal : 146 Fund(e) und 537 Messungen. Datierungen für 4 Perioden.
Pelvis : 34 Fund(e) und 58 Messungen. Datierungen für 3 Perioden.
Radius : 125 Fund(e) und 367 Messungen. Datierungen für 3 Perioden.
Scapula : 83 Fund(e) und 198 Messungen. Datierungen für 3 Perioden.
Second phalanx : 1 Fund(e) und 4 Messungen. Datierungen für 1 Perioden.
- DELETING Second phalanx  - not enough data
Tibia : 135 Fund(e) und 335 Messungen. Datierungen für 4 Perioden.
Ulna : 5 Fund(e)

Die Einträge mit weniger als 10 Funden und/oder weniger als 2 Perioden werden in der weiteren Betrachtung außer Acht gelassen, da sie nicht ausreichend Daten für einen Vergleich liefern. Dies betrifft die Einträge für "Mandible", "Second phalanx" und "Ulna".
Die Anzahl an Knochentypen reduziert sich damit von 14 auf 11.